# Running the strategy

In [21]:
!python /home/jovyan/work/q_pack/q_run/12_Clean_Strategy_BT.py

Profit ... or Loss: 32.62


In [22]:
! python /home/jovyan/work/q_pack/q_run/12_Clean_Strategy_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD \
--strat_param=model_uri=b5c9975d81c64bc3b662654007e1c20d,ml_serving=True

s3://mlflow-models/b5c9975d81c64bc3b662654007e1c20d/artifacts/model
Profit ... or Loss: -969.07


In [23]:
! python /home/jovyan/work/q_pack/q_run/12_Clean_Strategy_BT.py \
--strat_name=simple_strategy_2 \
--mode=backtest \
--tickers=EUR_USD,GBP_USD

Profit ... or Loss: 32.62


# Preprocessing the file

In [1]:
from ml_pack.preprocessing.ml_preprocessing import ml_preprocessing
ml_preprocessing(input_file="6_ml_log.csv")

'processed_6_ml_log.csv'

# Train a ML-model using MLflow

### Create an MLFlow experiment

In [35]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import mlflow
import mlflow.sklearn
import mlflow
import mlflow.pyfunc
import io
import boto3

In [36]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [37]:
# mlflow.create_experiment(name='simple_trading_ml_models', artifact_location='s3://mlflow-models')

In [38]:
mlflow.tracking.set_tracking_uri('http://mlflow-image:5500')
mlflow.set_experiment('simple_trading_ml_models')

In [39]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
data = pd.read_csv(s3.get_object(Bucket="model-support-files", Key="processed_6_ml_log.csv")['Body'],sep=',')

In [40]:
data.head()

,RSI,STOCHASTIC,fwd_returns
0,56.991619,78.882076,-0.005146
1,59.448606,77.279931,0.003864
2,60.945457,79.322084,0.013553
3,67.632318,67.385341,0.019120
4,56.546196,72.484186,-0.000088


In [41]:
# warnings.filterwarnings("ignore")
np.random.seed(40)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "fwd_returns"
train_x = train.drop(["fwd_returns"], axis=1)
test_x = test.drop(["fwd_returns"], axis=1)
train_y = train[["fwd_returns"]]
test_y = test[["fwd_returns"]]

n_estimators=200
max_depth=10

lr = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth)
lr.fit(train_x, train_y)
predicted_qualities = lr.predict(test_x)
(rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

print("RandomForest Model (n_estimators=%f, max_depth=%f):" % (n_estimators, max_depth))
print("  RMSE: %s" % rmse)
print("  MAE: %s" % mae)
print("  R2: %s" % r2)

mlflow.log_param("n_estimators", n_estimators)
mlflow.log_param("max_depth", max_depth)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)
mlflow.log_metric("mae", mae)
mlflow.sklearn.log_model(lr, "model")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForest Model (n_estimators=200.000000, max_depth=10.000000):
  RMSE: 0.009057421756148985
  MAE: 0.006710660830377815
  R2: 0.37834992926304445


## Running the strategy with the model

In [ ]:
!python /home/jovyan/work/q_pack/q_run/12_Clean_Strategy_BT.py \
--strat_name=simple_strategy_2 \
--strat_param=ml_serving=True,model_uri=b5c9975d81c64bc3b662654007e1c20d \
--ml_log=False \
--mode=backtest \
--tickers=EUR_USD

In [2]:
!python3 /home/jovyan/work/q_pack/q_run/12_Clean_Strategy_BT.py \
--strat_name=simple_strategy_2 \
--strat_param=ml_serving=True,model_uri="b5c9975d81c64bc3b662654007e1c20d" \
--ml_log=True \
--mode=backtest \
--tickers=EUR_USD

True
<class 'bool'>
ML Log Saved in Minio Bucket: model-support-files as 29_ml_log.csv
Profit ... or Loss: -515.62
